# TorchText (Old and New way)

In [20]:
import spacy, torch, json
from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

spacy_eng = spacy.load("en_core_web_sm")
spacy_ger = spacy.load("de_core_news_sm")


def tokenize_eng(text):
    return [tok.text for tok in spacy_eng.tokenizer(text)]


def tokenize_ger(text):
    return [tok.text for tok in spacy_ger.tokenizer(text)]


english = Field(sequential=True, use_vocab=True, tokenize=tokenize_eng, lower=True)
german = Field(sequential=True, use_vocab=True, tokenize=tokenize_ger, lower=True)

train_data, validation_data, test_data = Multi30k.splits(
    exts=(".de", ".en"), fields=(german, english)
)
#with open('../datasets/train.json', 'rb') as f:
#    train_data = json.load(f)
    
#with open('../datasets/validation.json', 'rb') as f:
#    validation_data = json.load(f)

english.build_vocab(train_data, max_size=10000, min_freq=2)
german.build_vocab(train_data, max_size=10000, min_freq=2)

train_iterator, validation_iterator = BucketIterator.splits(
    (train_data, validation_data), batch_size=64, device=device
)

for batch in train_iterator:
    print(batch)
    break

# string to integer (stoi)
print(f'Index of the word (the) is: {english.vocab.stoi["the"]}')

# print integer to string (itos)
print(f"Word of the index (1612) is: {english.vocab.itos[1612]}")
print(f"Word of the index (0) is: {english.vocab.itos[0]}")


[torchtext.legacy.data.batch.Batch of size 64 from MULTI30K]
	[.src]:[torch.LongTensor of size 20x64]
	[.trg]:[torch.LongTensor of size 27x64]
Index of the word (the) is: 5
Word of the index (1612) is: binoculars
Word of the index (0) is: <unk>


In [23]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import Multi30k

tokenize_ger = get_tokenizer('spacy', language='de_core_news_sm')
tokenize_eng = get_tokenizer('spacy', language='en_core_web_sm')
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']